In [1]:
import requests
import pandas as pd
import xml.etree.ElementTree as ET
import warnings
import math
from tqdm import tqdm
from irsx.xmlrunner import XMLRunner
xml_runner = XMLRunner()
warnings.filterwarnings('ignore')

import ipyparallel as ipp
c = ipp.Client()
print(c.ids)
dview = c[:] #So this dview (direct viewing) becomes the parallelizing program-direct execution in cloud

[0, 1, 2, 3]


In [2]:
einlist = pd.read_csv("/Users/Wolfalev/Documents/Grad/UT-AUSTIN GRAD/Coursework/CUSS/JiRepo/npo_classifier/dataset/eNGO_TX_ein_name_list.csv")

#for ein in einlist['EIN']:
 #   print(ein)

In [3]:
#xml_runner = XMLRunner()
#parsed_filing = xml_runner.run_sked(2012134249347, 'IRS990ScheduleJ')

### Need to get object ID

### Prepare UK Data.

```Python
# Compress icnpo training data.
df_icnpo_classifier_training_data=pd.read_csv('../dataset/icnpo_classifier_training_data.csv', sep=',')
df_icnpo_classifier_training_data.to_pickle('../dataset/icnpo_classifier_training_data.pkl.gzip', compression='gzip')
```

### Prepare US Training Data.
Useful links:
- IRS 990 forms on AWS: https://registry.opendata.aws/irs990/
- NCCS Data Archive (data dict, NTEE codes, etc.): https://nccs-data.urban.org/index.php

### TODO:
- [+] Add year of formation. - Do not use information from 990 forms: only Form 990 has "Year of Formation". Use BMF 199508 to dummy-code the organizations.
- [ ] Add program descriptions in Part VIII of Form 990.

In [4]:
df_index=pd.DataFrame()
for year in range(2013, 2019):
        try:
            print(str(year), 'started', end='\t')
            df_index_temp=pd.read_csv('https://s3.amazonaws.com/irs-form-990/index_'+str(year)+'.csv', 
                                  error_bad_lines=False, warn_bad_lines=True # Be cautious with bad lines.
                                 )
            df_index_temp['YEAR']=year
            df_index=pd.concat([df_index, df_index_temp], ignore_index=True)
            print('got')
        except:
            print('passed')

2013 started	got
2014 started	

b'Skipping line 39569: expected 9 fields, saw 10\n'


got
2015 started	got
2016 started	got
2017 started	got
2018 started	got


In [5]:
df_index.groupby(['YEAR','RETURN_TYPE']).count()

RETURN_ID  FILING_TYPE     EIN  TAX_PERIOD  SUB_DATE  \
YEAR RETURN_TYPE                                                         
2013 990             115531       115531  115531      115531    115531   
     990EO            21991        21991   21991       21991     21991   
     990EZ            60484        60484   60484       60484     60484   
     990O             38029        38029   38029       38029     38029   
     990PF            25414        25414   25414       25414     25414   
2014 990             163381       163381  163381      163381    163381   
     990EO            29466        29466   29466       29466     29466   
     990EZ            82937        82937   82937       82937     82937   
     990O             52490        52490   52490       52490     52490   
     990PF            59254        59254   59254       59254     59254   
2015 990             106571       106571  106571      106571    106571   
     990EO            21399        21399   21399       21399     21399   
     990EZ            59845        59845   59845       59845     59845   
     990O             34567        34567   34567       34567     34567   
     990PF            38650        38650   38650       38650     38650   
2016 990             160550       160550  160550      160550    160550   
     990EO            28537        28537   28537       28537     28537   
     990EZ            84652        84652   84652       84652     84652   
     990O             50987        50987   50987       50987     50987   
     990PF            53694        53694   53694       53694     53694   
2017 990             207734       207734  207734      207734    207734   
     990EO            36318        36318   36318       36318     36318   
     990EZ           109648       109648  109648      109648    109648   
     990O             66766        66766   66766       66766     66766   
     990PF            68547        68547   68547       68547     68547   
2018 990             192489       192489  192489      192489    192489   
     990EO            32690        32690   32690       32690     32690   
     990EZ           103694       103694  103694      103694    103694   
     990O             61005        61005   61005       61005     61005   
     990PF            67632        67632   67632       67632     67632   

                  TAXPAYER_NAME     DLN  OBJECT_ID  
YEAR RETURN_TYPE                                    
2013 990                 115531  115531     115531  
     990EO                21991   21991      21991  
     990EZ                60484   60484      60484  
     990O                 38029   38029      38029  
     990PF                25414   25414      25414  
2014 990                 163381  163381     163381  
     990EO                29466   29466      29466  
     990EZ                82937   82937      82937  
     990O                 52490   52490      52490  
     990PF                59254   59254      59254  
2015 990                 106571  106571     106571  
     990EO                21399   21399      21399  
     990EZ                59845   59845      59845  
     990O                 34567   34567      34567  
     990PF                38650   38650      38650  
2016 990                 160550  160550     160550  
     990EO                28537   28537      28537  
     990EZ                84652   84652      84652  
     990O                 50987   50987      50987  
     990PF                53694   53694      53694  
2017 990                 207734  207734     207734  
     990EO                36318   36318      36318  
     990EZ               109648  109648     109648  
     990O                 66766   66766      66766  
     990PF                68547   68547      68547  
2018 990                 192489  192489     192489  
     990EO                32690   32690      32690  
     990EZ               103694  103694     103694  
     990O                 61005   61005      61005  
     9

### Acquiring text data.

In [6]:
df_index_matched = df_index[df_index['EIN'].isin(einlist['EIN'])]
                       
len(df_index_matched)                                               

2458

In [7]:
df_index_matched.sample(10)

,RETURN_ID,FILING_TYPE,EIN,TAX_PERIOD,SUB_DATE,TAXPAYER_NAME,RETURN_TYPE,DLN,OBJECT_ID,YEAR
197951,10767997,EFILE,134249347,201212,8/14/2013,SOUTH TEXAS WATER UTILITY MANAGERS ASSOCIATION,990EO,93492135029973,201321359349202997,2013
1536157,14637133,EFILE,742974140,201612,8/11/2017 6:38:31 PM,VICTORIA COUNTY MASTER GARDENER ASSOCIATION,990EZ,93492066000147,201740669349200014,2017
1478066,14583916,EFILE,742590850,201609,7/25/2017 8:29:15 AM,KEEP HARLINGEN BEAUTIFUL INC,990EZ,93492041009617,201710419349200961,2017
1998171,15650684,EFILE,742974140,201712,9/4/2018 1:53:44 PM,VICTORIA COUNTY MASTER GARDENER ASSOCIATION,990EZ,93492127013068,201811279349201306,2018
1797237,15069835,EFILE,453732100,201612,1/2/2018 12:37:33 PM,TEXAS DESALINATION ASSOCIATION,990EO,93492317025867,201713179349202586,2018
232432,10723895,EFILE,562298497,201212,7/31/2013,JABO GREEN MINISTRIES INC,990EZ,93492143006173,201321439349200617,2013
521786,12037467,EFILE,741648152,201312,12/3/2014 5:34:49 PM,NORCHESTER MAINTENANCE FUND INC CO CHAPARRAL M...,990O,93493315024024,201423159349302402,2014
949790,13152730,EFILE,611535764,201412,01/27/2016,AUSTIN HUMAN RESOURCE MANAGEMENT ASSOCIATION,990O,93493295012855,201502959349301285,2016
697662,12638866,EFILE,760519231,201412,7/6/2015,FRIENDS OF ANAHUAC REFUGE,990,93493127020195,201541279349302019,2015
1532009,14677594,EFILE,752603041,201608,8/25/2017 9:44:28 AM,PINEY WOODS REGIONAL ADVISORY COUNC,990,93493103012237,201731039349301223,2017


In [8]:
done_index_list=[]
dview.execute('import pandas as pd')
dview.execute('from time import sleep')
dview.execute('from irsx.xmlrunner import XMLRunner')
dview.execute('import os')
dview['xml_runner'] = XMLRunner()
dview['df_index'] = df_index_matched
dview['df_index_text_acq'] = pd.DataFrame()

got unknown result: 75bf7770-d2a56134f5b8e4f9642cc581


In [9]:
dview['done_index_list'] = list(set(done_index_list))
dview['counter']=0

In [10]:
###### Define functions ######################################
def func_irs990_text(obj_id):
    IRS990_result=xml_runner.run_sked(obj_id, 'IRS990').result 

    ############## Check filing version.##############
    if IRS990_result!=None:
        IRS990_result=IRS990_result[0]
    else:
        return {'IRS990_p1_ActvtyOrMssnDsc': 'VERSION_NOT_SUPPORTED',
                'IRS990_p3_MssnDsc': 'VERSION_NOT_SUPPORTED',
                'IRS990_p3_DscS': 'VERSION_NOT_SUPPORTED',
                'IRS990ScheduleO_ExplntnTxt': 'VERSION_NOT_SUPPORTED',
               }
    ##################################################

    ###### IRS990_p1_ActvtyOrMssnDsc #################
    IRS990_p1_ActvtyOrMssnDsc=IRS990_result['schedule_parts']['part_i']['ActvtyOrMssnDsc']
    ##################################################

    ##### IRS990_p3_MssnDsc, IRS990_p3_DscS ##########
    IRS990_p3=IRS990_result['schedule_parts']['part_iii']
    IRS990_p3_MssnDsc='##'.join([IRS990_p3[key] for key in IRS990_p3.keys() if key=='MssnDsc'])
    IRS990_p3_DscS='##'.join([IRS990_p3[key] for key in IRS990_p3.keys() if (key=='Dsc' or '_Dsc' in key)])
    ##################################################

    ###### IRS990ScheduleO_ExplntnTxt #################
#         if 'InfInSkdOPrtIIIInd' in IRS990_p3.keys() and IRS990_p3['InfInSkdOPrtIIIInd']=='X' and 'IRS990ScheduleO' in xml_runner.run_filing(obj_id).list_schedules():
    if 'IRS990ScheduleO' in xml_runner.run_filing(obj_id).list_schedules():
        IRS990ScheduleO=xml_runner.run_sked(obj_id, 'IRS990ScheduleO').result[0]
        IRS990ScheduleO_ExplntnTxt='##'.join([s['ExplntnTxt'] for s in IRS990ScheduleO['groups']['SkdOSpplmntlInfrmtnDtl'] if 'III' in s['FrmAndLnRfrncDsc']])
    else:
        IRS990ScheduleO_ExplntnTxt=''
    ##################################################

    return {'IRS990_p1_ActvtyOrMssnDsc': IRS990_p1_ActvtyOrMssnDsc,
            'IRS990_p3_MssnDsc': IRS990_p3_MssnDsc,
            'IRS990_p3_DscS': IRS990_p3_DscS,
            'IRS990ScheduleO_ExplntnTxt': IRS990ScheduleO_ExplntnTxt,
           }

def func_irs990ez_text(obj_id):
    IRS990EZ_result=xml_runner.run_sked(obj_id, 'IRS990EZ').result

    ############## Check filing version.##############
    if IRS990EZ_result!=None:
        IRS990EZ_result=IRS990EZ_result[0]
    else:
        return {'IRS990EZ_p3_PrmryExmptPrpsTxt': 'VERSION_NOT_SUPPORTED',
                'IRS990EZ_p3_DscrptnPrgrmSrvcAccmTxt': 'VERSION_NOT_SUPPORTED',
                'IRS990ScheduleO_ExplntnTxt': 'VERSION_NOT_SUPPORTED',
               }
    ########################################################

    ###### IRS990EZ_p3_PrmryExmptPrpsTxt #############
    if 'ez_part_iii' in IRS990EZ_result['schedule_parts'].keys():
        IRS990EZ_p3=IRS990EZ_result['schedule_parts']['ez_part_iii']
        IRS990EZ_p3_PrmryExmptPrpsTxt='##'.join([IRS990EZ_p3[key] for key in IRS990EZ_p3.keys() if key=='PrmryExmptPrpsTxt'])
    else:
        IRS990EZ_p3_PrmryExmptPrpsTxt=''
    ##################################################

    ####### IRS990EZ_p3_DscrptnPrgrmSrvcAccmTxt ######
    if 'EZPrgrmSrvcAccmplshmnt' in IRS990EZ_result['groups'].keys():
        IRS990EZ_p3_DscrptnPrgrmSrvcAccmTxt='##'.join([s['DscrptnPrgrmSrvcAccmTxt'] for s in IRS990EZ_result['groups']['EZPrgrmSrvcAccmplshmnt']])
    else:
        IRS990EZ_p3_DscrptnPrgrmSrvcAccmTxt=''
    ##################################################

    ###### IRS990ScheduleO_ExplntnTxt #################
#         if 'InfInSkdOPrtIIIInd' in IRS990EZ_p3.keys() and IRS990EZ_p3['InfInSkdOPrtIIIInd']=='X' and 'IRS990ScheduleO' in xml_runner.run_filing(obj_id).list_schedules():
    if 'IRS990ScheduleO' in xml_runner.run_filing(obj_id).list_schedules():
        IRS990ScheduleO=xml_runner.run_sked(obj_id, 'IRS990ScheduleO').result[0]
        IRS990ScheduleO_ExplntnTxt='##'.join([s['ExplntnTxt'] for s in IRS990ScheduleO['groups']['SkdOSpplmntlInfrmtnDtl'] if 'III' in s['FrmAndLnRfrncDsc']])
    else:
        IRS990ScheduleO_ExplntnTxt=''
    ##################################################

    return {'IRS990EZ_p3_PrmryExmptPrpsTxt': IRS990EZ_p3_PrmryExmptPrpsTxt,
            'IRS990EZ_p3_DscrptnPrgrmSrvcAccmTxt': IRS990EZ_p3_DscrptnPrgrmSrvcAccmTxt,
            'IRS990ScheduleO_ExplntnTxt': IRS990ScheduleO_ExplntnTxt,
           }

def func_irs990pf_text(obj_id):
    IRS990PF_result=xml_runner.run_sked(obj_id, 'IRS990PF').result

    ############## Check filing version.##############
    if IRS990PF_result!=None:
        IRS990PF_result=IRS990PF_result[0]
    else:
        return {'IRS990PF_p9a_DscrptnTxt': 'VERSION_NOT_SUPPORTED',
                'IRS990PF_p16b_RltnshpSttmntTxt': 'VERSION_NOT_SUPPORTED',
               }
    ########################################################

    ##### IRS990PF_p9a_DscrptnTxt ##########################
    if 'pf_part_ixa' in IRS990PF_result['schedule_parts'].keys():
        IRS990PF_p9a=IRS990PF_result['schedule_parts']['pf_part_ixa']
        IRS990PF_p9a_DscrptnTxt='##'.join([IRS990PF_p9a[key] for key in IRS990PF_p9a.keys() if 'Txt' in key])
    else:
        IRS990PF_p9a_DscrptnTxt=''
    ########################################################

    ##### IRS990PF_p16b_RltnshpSttmntTxt ###################
    if 'PFRlnOfActyTAccmOfExmptPrps' in IRS990PF_result['groups']:
        IRS990PF_p16b_RltnshpSttmntTxt='##'.join([s['RltnshpSttmntTxt'] for s in IRS990PF_result['groups']['PFRlnOfActyTAccmOfExmptPrps']])
    else:
        IRS990PF_p16b_RltnshpSttmntTxt=''
    ########################################################

    return {'IRS990PF_p9a_DscrptnTxt':IRS990PF_p9a_DscrptnTxt,
            'IRS990PF_p16b_RltnshpSttmntTxt':IRS990PF_p16b_RltnshpSttmntTxt,
           }
###### Define functions ######################################

In [11]:
###### Run main function ################################
df_index_text_acq = pd.DataFrame()
df_index = pd.DataFrame()
df_index = df_index_matched
done_index_list = []
for index in df_index_matched.index:
    try:
        if index not in done_index_list: #Index is position of filing in list of all filings in index file
            obj_id = df_index.loc[index, 'OBJECT_ID'] #if that position is not filled, pull the object id
            return_type=df_index.loc[index, 'RETURN_TYPE'] #if that position is not filled, pull the return type
            if return_type in ['990', '990O']: # (1) if return type  is 990, do the following
                text_dict=func_irs990_text(obj_id) #apply this function for 990s, using the object id from index to get the 990 in a text_dict
                df_index.loc[index, 'IRS990_p1_ActvtyOrMssnDsc']=text_dict['IRS990_p1_ActvtyOrMssnDsc'] #get mission and put it in index file
                df_index.loc[index, 'IRS990_p3_MssnDsc']=text_dict['IRS990_p3_MssnDsc'] # get mssndsc and put it here
                df_index.loc[index, 'IRS990_p3_DscS']=text_dict['IRS990_p3_DscS'] # get DsC and put it here
                df_index.loc[index, 'IRS990ScheduleO_ExplntnTxt']=text_dict['IRS990ScheduleO_ExplntnTxt']# get schedule 0 and put it here
            elif return_type in ['990EZ', '990EO']: #(2) same thing
                text_dict=func_irs990ez_text(obj_id)
                df_index.loc[index, 'IRS990EZ_p3_PrmryExmptPrpsTxt']=text_dict['IRS990EZ_p3_PrmryExmptPrpsTxt']
                df_index.loc[index, 'IRS990EZ_p3_DscrptnPrgrmSrvcAccmTxt']=text_dict['IRS990EZ_p3_DscrptnPrgrmSrvcAccmTxt']
                df_index.loc[index, 'IRS990ScheduleO_ExplntnTxt']=text_dict['IRS990ScheduleO_ExplntnTxt']
            elif return_type in ['990PF']: #(3) same thing
                text_dict=func_irs990pf_text(obj_id)
                df_index.loc[index, 'IRS990PF_p9a_DscrptnTxt']=text_dict['IRS990PF_p9a_DscrptnTxt']
                df_index.loc[index, 'IRS990PF_p16b_RltnshpSttmntTxt']=text_dict['IRS990PF_p16b_RltnshpSttmntTxt']
            df_index_text_acq=pd.concat([df_index_text_acq, df_index.loc[[index]]], ignore_index=True) #take the entries we just pulled, turn into df for that row, and concatenate them together into df_index_text_acq
            done_index_list+=[index] ## update that we've done this many indicies. 
    except:
        pass

Filing version 2012v2.1 isn't supported for this operation
Filing version 2012v2.1 isn't supported for this operation
Filing version 2012v2.1 isn't supported for this operation
Filing version 2012v2.1 isn't supported for this operation
Filing version 2012v2.1 isn't supported for this operation
Filing version 2011v1.5 isn't supported for this operation
Filing version 2012v2.1 isn't supported for this operation
Filing version 2011v1.5 isn't supported for this operation
Filing version 2012v2.1 isn't supported for this operation
Filing version 2012v2.0 isn't supported for this operation
Filing version 2012v2.0 isn't supported for this operation
Filing version 2011v1.3 isn't supported for this operation
Filing version 2012v2.1 isn't supported for this operation
Filing version 2012v2.2 isn't supported for this operation
Filing version 2011v1.5 isn't supported for this operation
Filing version 2012v2.0 isn't supported for this operation
Filing version 2012v2.3 isn't supported for this operati

In [12]:
len(set(done_index_list)), len(df_index_text_acq)

(2455, 2455)

In [20]:
df_index_text_acq.sample(10)

2455


In [18]:
df_index_text_acq_2018= df_index_text_acq[df_index_text_acq.YEAR == 2018]

In [21]:
df_index_text_acq_2018.sample(10)
print(len(df_index_text_acq_2018))

487


len(df_EIN_TXT), len(df_index_matched)

In [80]:
df_bmf_1812=pd.read_csv('https://nccs-data.urban.org/data/bmf/2018/bmf.bm1812.csv')
df_ein_18=df_bmf_1812
df_ein_18['2018']=1
df_ein_18 = df_ein_18[df_ein_18.STATE == 'TX']
df_ein_18 = df_ein_18[df_ein_18.NTEE1 == 'C']



In [81]:
df_bmf_1812['EIN'] = pd.to_numeric(df_bmf_1812['EIN'])
df_index_text_acq_2018['EIN'] = pd.to_numeric(df_index_text_acq_2018['EIN'])

In [82]:
df_bmf_1812.drop_duplicates(subset='EIN',keep='first')
print(len(df_bmf_1812))
df_index_text_acq_2018.drop_duplicates(subset='EIN',keep='first')
print(len(df_index_text_acq_2018))

1499450
487


In [83]:
df_EIN_TXT=df_index_text_acq_2018.merge(df_ein_18, on='EIN', how='outer')
df_EIN_TXT['2018']=df_EIN_TXT['2018'].fillna(0)
df_EIN_TXT.sample(3)

,DLN,EIN,FILING_TYPE,IRS990EZ_p3_DscrptnPrgrmSrvcAccmTxt,IRS990EZ_p3_PrmryExmptPrpsTxt,IRS990PF_p16b_RltnshpSttmntTxt,IRS990PF_p9a_DscrptnTxt,IRS990ScheduleO_ExplntnTxt,IRS990_p1_ActvtyOrMssnDsc,IRS990_p3_DscS,...,LEVEL1,NTMAJ10,MAJGRPB,LEVEL3,LEVEL2,NTMAJ12,NTMAJ5,FILER,ZFILER,2018
142,9.349303e+13,510187748,EFILE,NaN,NaN,NaN,NaN,,TO PROMOTE MEMBER SERVICES REGARDING HORTICULTURE,PROMOTES NUMEROUS TOURS AND LECTURES DURING TH...,...,O,EN,C,EN,O,EN,OT,Y,N,1.0
972,NaN,581946142,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,O,EN,C,EN,O,EN,OT,N,N,1.0
337,9.349319e+13,742924909,EFILE,NaN,NaN,NaN,NaN,THE COASTAL BEND BAYS & ESTUARIES PROGRAM FUNC...,COASTAL BEND BAYS & ESTUARIES PROGAM IS DEDICA...,THE U.S. ENVIRONMENTAL PROTECTION AGENCY'S (EP...,...,PC,EN,C,EN,O,EN,OT,Y,N,1.0


In [86]:
df_EIN_TXT.columns

Index(['DLN', 'EIN', 'FILING_TYPE', 'IRS990EZ_p3_DscrptnPrgrmSrvcAccmTxt',
       'IRS990EZ_p3_PrmryExmptPrpsTxt', 'IRS990PF_p16b_RltnshpSttmntTxt',
       'IRS990PF_p9a_DscrptnTxt', 'IRS990ScheduleO_ExplntnTxt',
       'IRS990_p1_ActvtyOrMssnDsc', 'IRS990_p3_DscS', 'IRS990_p3_MssnDsc',
       'OBJECT_ID', 'RETURN_ID', 'RETURN_TYPE', 'SUB_DATE', 'TAXPAYER_NAME',
       'TAX_PERIOD', 'YEAR', 'SEC_NAME', 'FRCD', 'SUBSECCD', 'TAXPER',
       'ASSETS', 'INCOME', 'NAME', 'ADDRESS', 'CITY', 'STATE', 'NTEECONF',
       'NTEEFINAL', 'NAICS', 'ZIP5', 'OUTNCCS', 'OUTREAS', 'RULEDATE', 'FIPS',
       'FNDNCD', 'PMSA', 'MSA_NECH', 'CASSETS', 'CFINSRC', 'CTAXPER',
       'CTOTREV', 'ACCPER', 'RANDNUM', 'NTEECC', 'NTEE1', 'LEVEL4', 'LEVEL1',
       'NTMAJ10', 'MAJGRPB', 'LEVEL3', 'LEVEL2', 'NTMAJ12', 'NTMAJ5', 'FILER',
       'ZFILER', '2018'],
      dtype='object')

In [87]:
print(len(df_EIN_TXT))

1631


In [88]:
df_EIN_TXT['EIN'].unique

<bound method Series.unique of 0       471982274
1       471982274
2       741696145
3       741696145
4       742613666
5       741708558
6       200930296
7       200930296
8       742821500
9       742821500
10      461195052
11       61834089
12       61834089
13      453929881
14      760243519
15      751947699
16      751947699
17      742441468
18      742524314
19      237109965
20      900648270
21      900648270
22      760031884
23      760031884
24      743020979
25      743020979
26      204467651
27      742121275
28      742121275
29      471468601
          ...    
1601    830434834
1602    830462989
1603    850284313
1604    861067178
1605    861157331
1606    870744143
1607    870754471
1608    900357998
1609    900391541
1610    900453229
1611    900505937
1612    900520995
1613    900536009
1614    900601557
1615    900770416
1616    900793319
1617    900889748
1618    900986516
1619    901021494
1620    911632336
1621    911799412
1622    911917835
1623    9119646

In [89]:
df_EIN_TXT['EIN'].isin(einlist['EIN']).value_counts()

False    929
True     702
Name: EIN, dtype: int64

In [90]:
df_EIN_TXT.to_csv('./texas_2018_envorgs.csv', index=False)